--- Day 5: Binary Boarding ---
You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.

You write a quick program to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.

Instead of zones or groups, this airline uses binary space partitioning to seat people. A seat might be specified like FBFBBFFRLR, where F means "front", B means "back", L means "left", and R means "right".

The first 7 characters will either be F or B; these specify exactly one of the 128 rows on the plane (numbered 0 through 127). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the front (0 through 63) or the back (64 through 127). The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.

For example, consider just the first seven characters of FBFBBFFRLR:

Start by considering the whole range, rows 0 through 127.
F means to take the lower half, keeping rows 0 through 63.
B means to take the upper half, keeping rows 32 through 63.
F means to take the lower half, keeping rows 32 through 47.
B means to take the upper half, keeping rows 40 through 47.
B keeps rows 44 through 47.
F keeps rows 44 through 45.
The final F keeps the lower of the two, row 44.
The last three characters will be either L or R; these specify exactly one of the 8 columns of seats on the plane (numbered 0 through 7). The same process as above proceeds again, this time with only three steps. L means to keep the lower half, while R means to keep the upper half.

For example, consider just the last 3 characters of FBFBBFFRLR:

Start by considering the whole range, columns 0 through 7.
R means to take the upper half, keeping columns 4 through 7.
L means to take the lower half, keeping columns 4 through 5.
The final R keeps the upper of the two, column 5.
So, decoding FBFBBFFRLR reveals that it is the seat at row 44, column 5.

Every seat also has a unique seat ID: multiply the row by 8, then add the column. In this example, the seat has ID 44 * 8 + 5 = 357.

Here are some other boarding passes:

BFFFBBFRRR: row 70, column 7, seat ID 567.
FFFBBBFRRR: row 14, column 7, seat ID 119.
BBFFBBFRLL: row 102, column 4, seat ID 820.
As a sanity check, look through your list of boarding passes. What is the highest seat ID on a boarding pass?

Your puzzle answer was 996.

The first half of this puzzle is complete! It provides one gold star: *

In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = 'C:/Users\Loren/Desktop/advent_of_code_day5.txt'

In [3]:
df = pd.read_csv(file_path, header=None)
df = df.rename(columns={0:'seat_spec'})

In [4]:
df.head()

,seat_spec
0,BBFFBFBRLL
1,FFFFBFBRLR
2,BFFBBFBRLR
3,BFBBBFBLLL
4,FFBBFBBLRR


In [5]:
#deal with first 7
def front_back(seat_spec):
    
    first_7 = str(seat_spec[:7]) #first 7 chars

    start_row = 0
    end_row = 127
    
    for i in first_7:
        if i == 'F': # front - keep lower(front) half, move end in
            end_row = end_row - int(len(range(start_row, end_row + 1))/2)   
        elif i == 'B': # back - keep upper(back) half, move start up
            start_row = start_row + int(len(range(start_row, end_row + 1))/2)   
    return start_row

df['front_back'] = df.apply(lambda x: front_back(x['seat_spec']), axis=1)    

In [6]:
#deal with last 3
def left_right(seat_spec):
    
    last_3 = str(seat_spec[-3:]) # last 3 chars

    start_seat = 0
    end_seat = 7
    
    for i in last_3:
        if i == 'L': # front - keep lower(front) half, move end in
            end_seat = end_seat - int(len(range(start_seat, end_seat + 1))/2)   
        elif i == 'R': # back - keep upper(back) half, move start up
            start_seat = start_seat + int(len(range(start_seat, end_seat + 1))/2)   
    return start_seat
    
df['left_right'] = df.apply(lambda x: left_right(x['seat_spec']), axis=1)    

In [7]:
#seatID = multiply FB by 8 then add LR
df['seat_id'] = df['front_back'] * 8 + df['left_right']

In [8]:
df.head()

,seat_spec,front_back,left_right,seat_id
0,BBFFBFBRLL,101,4,812
1,FFFFBFBRLR,5,5,45
2,BFFBBFBRLR,77,5,621
3,BFBBBFBLLL,93,0,744
4,FFBBFBBLRR,27,3,219


In [9]:
#find test example in 'my input' and check
df.loc[df['seat_spec'] == 'FBFBBFFRLR']

,seat_spec,front_back,left_right,seat_id
143,FBFBBFFRLR,44,5,357


In [10]:
#find test example in 'my input' and check
df.loc[df['seat_spec'] == 'BFFFBBFRRR']

,seat_spec,front_back,left_right,seat_id
626,BFFFBBFRRR,70,7,567


In [11]:
#find test example in 'my input' and check
df.loc[df['seat_spec'] == 'FFFBBBFRRR']

,seat_spec,front_back,left_right,seat_id
432,FFFBBBFRRR,14,7,119


In [12]:
#find test example in 'my input' and check
df.loc[df['seat_spec'] == 'BBFFBBFRLL']

,seat_spec,front_back,left_right,seat_id
506,BBFFBBFRLL,102,4,820


In [13]:
#answer - max seat ID
df['seat_id'].max()

996

Part 2

--- Part Two ---
Ding! The "fasten seat belt" signs have turned on. Time to find your seat.

It's a completely full flight, so your seat should be the only missing boarding pass in your list. However, there's a catch: some of the seats at the very front and back of the plane don't exist on this aircraft, so they'll be missing from your list as well.

Your seat wasn't at the very front or back, though; the seats with IDs +1 and -1 from yours will be in your list.

What is the ID of your seat?

Your puzzle answer was 671.

Both parts of this puzzle are complete! They provide two gold stars: **

In [14]:
df.sort_values(by='seat_id', inplace=True)

In [15]:
df.reset_index(inplace=True) 
df.index += df['seat_id'].min() #start index at first seat ID

In [16]:
df.reset_index(inplace=True) #reset index again to have index starting at min seats part of calcs

In [17]:
df.rename(columns={'level_0':'seat_id_ref'}, inplace=True) 

In [18]:
df['check_seat_id_matches_ref'] = df['seat_id'] - df['seat_id_ref']

In [19]:
df.head()

,seat_id_ref,index,seat_spec,front_back,left_right,seat_id,check_seat_id_matches_ref
0,21,147,FFFFFBFRLR,2,5,21,0
1,22,874,FFFFFBFRRL,2,6,22,0
2,23,42,FFFFFBFRRR,2,7,23,0
3,24,359,FFFFFBBLLL,3,0,24,0
4,25,784,FFFFFBBLLR,3,1,25,0


In [23]:
#find where mismatch happens
mismatch_check = df['check_seat_id_matches_ref'] == 1
df.loc[mismatch_check].head() # first seat ID ref with mismatched seat_id (sorted asc) == YOUR SEAT

,seat_id_ref,index,seat_spec,front_back,left_right,seat_id,check_seat_id_matches_ref
650,671,961,BFBFBFFLLL,84,0,672,1
651,672,76,BFBFBFFLLR,84,1,673,1
652,673,955,BFBFBFFLRL,84,2,674,1
653,674,827,BFBFBFFLRR,84,3,675,1
654,675,733,BFBFBFFRLL,84,4,676,1
